# Motivation

- Process huge datasets in batch 
- Online (streaming) algorithms that run in little memory
- Approximate answers must be good enough

## Typical tasks

- approximate counts
- approximate count distinct
- approximate membership in sets
- approximate intersection of sets (e.g. Jaccard Similarity)
- approximate nearest neighbor
- machine learning: fitting regressions with extremely large number of features
- multiplying huge matrices (approximately)
- large scale Singular Value Decomposition
- approximate numeric histograms
- approximate "heavy hitters" (most frequent items)

## Hash tables

Many of those algorithms are related to hash tables

## Excercise 1

Specify the interface of a hash table (write a class with methods without implementation)

In [22]:
class myHash:
    def __init__(self):
        pass
    
    def getHash(self,hash):
        pass

    def addHash(self,hash,el):
        pass

## Excercise 2

Fill-in the implementation of a hash table. The hash table will not store more than 1024 unique elements. Use lists of lists

In [1]:
class HashTable(object):
    def __init__(self, memorysize=1024):
        self._size = memorysize
        self._bucket = [[] for _ in range(self._size)]
        self._keys = []

    def get(self, key):
        i = self._get_index(key)
        for listy in self._bucket[i]:
            if listy[0] == key:
                return listy[1]
    
    def put(self, key, value):
        assert (not key in self._keys), "This key already exists in table"
        i = self._get_index(key)
        self._bucket[i].append([key,value])
        self._keys.append(key)
    
    def items(self):
        return [(key, self.get(key)) for key in self._keys]
    
    def _get_index(self, key):
        assert isinstance(key, str), "This implementation only accepts strings as keys"
        return hash(key) % self._size
    
# Table that collides
table = HashTable(memorysize=2)
table.put('Hello', "world")
table.put('Bonjour', "monde")
table.put('Hola', "mundo")
table.items()

[('Hello', 'world'), ('Bonjour', 'monde'), ('Hola', 'mundo')]

## Excercise 3

Explore collections.defaultdict to count (key,values). Can you use a counter.




In [2]:
from collections import defaultdict
dd = defaultdict(list)
for k,v in table.items():
    dd[k].append(v)
len(list(dd.values()))

3

## Excercise 4
Count the number of distinct objects in a list

In [23]:
# Based on http://stackoverflow.com/questions/2257441/random-string-generation-with-upper-case-letters-and-digits-in-python
import string
import random
def random_string(size=20, chars=string.ascii_uppercase + string.digits):
    return ''.join(random.choice(chars) for _ in range(size))

def seq_of_random_strings(n):
    for i in range(0, n):
        yield random_string()
    
random_strings = seq_of_random_strings(1000)
random_strings

<generator object seq_of_random_strings at 0x7f59904b0e08>

In [4]:
for i,s in enumerate(random_strings):
    if i >= 10:
        break
    print(s)
    
print(len(set(random_strings)))


GB20387W6RJ36U13W7C4
L6BMWPI3TG80LOWGC1ZZ
3SRXQBZKVX38IDK9306L
1L411U55JP8ACYACSLJJ
DPIMR7IOVFU0KXGDBTOL
NH7P1GGRSH9DZ8ETDU4A
OO6T3UI96M2IA70Q63F9
Z4FL5RCY902V4BJNQR7N
F2AB2BMAVUV9I6VZPRZO
47YDTH8JXLGW8X60X3YC
989


In [12]:
# Accurate but very resource consuming as the lists grows
random_strings = seq_of_random_strings(10**6)
len(set(random_strings))

1000000

In [17]:
import hyperloglog
hll = hyperloglog.HyperLogLog(0.01)

random_strings = seq_of_random_strings(10**6)
for s in random_strings:
    hll.add(s)

#print(len(hll))

In [18]:
%time print(len(hll))

995169
CPU times: user 8 ms, sys: 0 ns, total: 8 ms
Wall time: 6.9 ms


## Excercise 5

Put the random strings in a Spark dataframe. Make sure to save and reload the Spark Dataframe.

- Use count distinct to count the number of unique strings
- Use count approx distinct to count the number of unique strings
- Report how much time and memory each of the methods took
- How different are the results

In [10]:
import hyperloglog
hll = hyperloglog.HyperLogLog(0.01)  # accept 1% counting error

In [11]:
for item in random_strings:
    hll.add(item)
print len(hll)  # 2

1001


In [44]:
def hllItems(items):
    hll = hyperloglog.HyperLogLog(0.01)  # accept 1% counting error
    for item in items:
        hll.add(item)
    return hll

set1 = [str(i) for i in range(0, 10000)]

set2 = [str(i) for i in range(5000, 15000)]

#unionSize = hllItems(set1 + set2)
#print len(unionSize)

hllS1 = hllItems(set1)
szS1 = len(hllS1)


hllS2 = hllItems(set2)
szS2 = len(hllS2)
print szS1, szS2


hllS1.update(hllS2) #equivalent to hllItems(set1 + set2)
print 'union', len(hllS1)
print 'interesection', szS1 + szS2 - len(hllS1)

9989 10006
union 15021
interesection 4974


## Excercise 6

Generate a dataset of key-value pairs and put it into a Spark dataframe.
For each unique key, count the number of unique values.

In [15]:
random_keys = list(seq_of_random_strings(1000))
random_values = list(seq_of_random_strings(100000))

def random_key_values(n):
    for i in xrange(0, n):
        random_key = random.choice(random_keys)
        random_value = random.choice(random_values)
        yield random_key, random_value

random_key_values(1000)

<generator object random_key_values at 0x7f82a01f3190>

## The HyperLogLog Algorithm

- Show slides

## Exercise 7

Find out which of the following database systems implements approximate counting via HyperLogLog or HyperLogLogPlusPlus

- Elasticsearch
- Redis
- Redshift
- MongoDB
- MySQL

## Bloom Filters


## Exercise 8

You have a file with longs from 1 to 100000000. However, about 10000 numbers are missing from this list.
If each long takes 64 bits the file is about 800 MBs. 
Can you figure out the missing numbers by reading the file only once and using less than 15 MB.

(Imagine you are using something like bitstring.BitArray)

In [15]:
input = [8,7,1,3,9]
array = [False] * 10
for v in input:
    array[v] = True
for i in range(1,10):
    if array[i] == False:
        print(i)

2
4
5
6


In [19]:
import sys
print(sys.getsizeof(array), sys.getsizeof(input))

144 104


In [20]:
# If implemented with bit array, the array size will be 64 times smaller than the input size containing longs
from bitstring import BitArray

In [32]:
class BloomFilter:
    def __init__(self, size):
        self.store = [0]*size
        self.size = size
        self.num_coll = 0
    
    def hashes(self, key):
        h1 = hash(key) % self.size
        h2 = hash((1 + key)*13) % self.size
        return (h1, h2)
    
   
    def add(self, key):
        h1, h2 = self.hashes(key)
        #if self.store[h1] == 1:
        #    print 'collision 1'
        #if self.store[h2] == 1:
        #    print 'collision 2' 
        if self.store[h1] == 1 and self.store[h2] == 1:
            #print 'collision 1/2'
            self.num_coll += 1
        self.store[h1] = 1
        self.store[h2] = 1
        pass
    
    def exists(self,key):
        h1, h2 = self.hashes(key)
        return self.store[h1] == 1 and self.store[h2] == 1
    
b = BloomFilter(4000)
for i in reversed(range(0, 1000)):
    b.add(i)
    
b.exists(777)
print b.num_coll
print b.exists(777)
print b.exists(7777)

27
True
False


## Exercise 9

You have 100000000 strings. A string represents a tuple (user_id, page_id), but this is not important.
You want to design a store such that supports:
    
    - check if a (user_id, page_id) is in the store
    - add a (user_id,page_id) to the store
    - use very little memory (i.e. bit arrays)
    - in 1 out 100 cases the data structure is allowed to report falsely that (user_id, page_id) is in the store


In [129]:
# Version with 2 hash function

from bitstring import BitArray
big_number = 10**4
factor = 8
random_strings = seq_of_random_strings(big_number)
ba = BitArray(length = big_number*factor)
ba.set(0)

# Used to store some of the strings to test the algorithm
some_strings=[]

# Populate the bit arrays
for c,s in enumerate(random_strings):
    i = hash(s) % (big_number * factor)
    ba[i] = True   
    if c % (big_number/10) == 0:
        some_strings.append(s)
 
# Check collisions
random_strings2 = seq_of_random_strings(big_number)
res = []
for s in random_strings2:
    s = s+"a"
    i = hash(s) % (big_number * factor)
    res.append(ba[i])

print("Collisions %.2f " % (100*sum(res)/big_number))

# Check collisions (100 times)
ress = []
for _ in range(10**2):
    random_strings2 = seq_of_random_strings(big_number)
    res = []
    for s in random_strings2:
        s = s+"a"
        i = hash(s) % (big_number * factor)
        res.append(ba[i])
    ress.append(np.mean(100*sum(res)/big_number))
np.mean(ress)

Collisions 11.77 


11.739899999999999

In [128]:
# Version with 2 hash functions

from bitstring import BitArray
big_number = 10**4
factor = 4
salt = "DSR" # Used to change the hash function

random_strings = seq_of_random_strings(big_number)
ba1 = BitArray(length = big_number*factor)
ba1.set(0)
ba2 = BitArray(length = big_number*factor)
ba2.set(0)

# Used to store some of the strings to test the algorithm
some_strings=[]

# Populate the bit arrays
for c,s in enumerate(random_strings):
    i = hash(s) % (big_number * factor)
    j = hash(s+salt) % (big_number *factor)
    ba1[i] = True 
    ba2[j] = True
    if c % (big_number/10) == 0:
        some_strings.append(s)

# Check collisions (1 trial)
random_strings2 = seq_of_random_strings(big_number)
res = []
for s in random_strings2:
    s = s+"a"
    i = hash(s) % (big_number * factor)
    j = hash(s+salt) % (big_number * factor )
    res.append(ba1[i] & ba2[j])

print("Collisions %.2f " % (100*sum(res)/big_number))

# Check collisions (100 trials)
ress = []
for _ in range(10**2):
    random_strings2 = seq_of_random_strings(big_number)
    res = []
    for s in random_strings2:
        s = s+"a"
        i = hash(s) % (big_number * factor)
        j = hash(s+salt) % (big_number * factor )
        res.append(ba[i] & ba[j])
    ress.append(np.mean(100*sum(res)/big_number))
np.mean(ress)

Collisions 4.79 


4.8856000000000002

In [127]:
# Version with 2 hash functions but only 1 array

import numpy as np

from bitstring import BitArray
big_number = 10**4
factor = 8
salt = "DSR" # Used to change the hash function

random_strings = seq_of_random_strings(big_number)
ba = BitArray(length = big_number*factor)
ba.set(0)

# Used to store some of the strings to test the algorithm
some_strings=[]

# Populate the bit arrays
for c,s in enumerate(random_strings):
    i = hash(s) % (big_number * factor)
    j = hash(s+salt) % (big_number *factor)
    ba[i] = True 
    ba[j] = True
    if c % (big_number/10) == 0:
        some_strings.append(s)

# Check collisions (1 trial)
random_strings2 = seq_of_random_strings(big_number)
res = []
for s in random_strings2:
    s = s+"a"
    i = hash(s) % (big_number * factor)
    j = hash(s+salt) % (big_number * factor )
    res.append(ba[i] & ba[j])

print("Collisions %.2f " % (100*sum(res)/big_number))
        
# Check collisions (100 trials)
ress = []
for _ in range(10**2):
    random_strings2 = seq_of_random_strings(big_number)
    res = []
    for s in random_strings2:
        s = s+"a"
        i = hash(s) % (big_number * factor)
        j = hash(s+salt) % (big_number * factor )
        res.append(ba[i] & ba[j])
    ress.append(np.mean(100*sum(res)/big_number))
np.mean(ress)

Collisions 4.98 


4.8587999999999996

## Exercise 10

Add the data random_strings from Excercise 4 to a BloomFilter
- check if each of the strings is correctly reported to be in the Bloom filter
- find a string on which the Bloom Filter makes a mistake

## Exercise 11

Generate two huge random datasets (based on random_strings). Store them in dataframes saved on disk.
- Use inner join in Spark to find the elements in both dataframes
- Use bloom filters in Spark to find the elements in both dataframes

## Exercise 12

Same as excercise 11, but use HyperLogLog to find only the intersection size of the inner join

# Sampling

## Excercise 13

Implement a function that shuffles a list of items (like random.shuffle) calls to random.randint(a,b)

## Excercise 14

Implement sampling on streams. This is called reservoir sampling. See below for the interface.

In [31]:
class ReservoirSampling:
    def __init__(self, k):
        """
            k: number of sample the 
        """
        self.k = k
        
    def maybe_add(self, value):
        """
            Add a value to the sample
        """
        pass
    
    def current_sample(self):
        """
            returns the current sample
        """
        pass    
    
    def merge(self, other):
        """
            merges two samples
            don't implement it here
        """
        pass

## Excercise 15

You have a stream of records (e.g. requests to web services) that have a value associated with them (e.g. latency).
The requests are streamed to a service that has to keep K (e.g. 1000) requests from the last day that have had highest latency.
